# PPG analysis

In [22]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import os

from paradigma.preprocessing_config import PPGPreprocessingConfig, IMUPreprocessingConfig
from paradigma.ppg_preprocessing import scan_and_sync_segments, preprocess_ppg_data
from paradigma.heart_rate_analysis_config import SignalQualityFeatureExtractionConfig, SignalQualityClassificationConfig
from paradigma.heart_rate_analysis import extract_signal_quality_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Cell has the tag 'parameters'
path_to_data =  '../../../tests/data'
path_to_classifier = '../../../src/paradigma/ppg/classifier'

quality_classifier_path = os.path.join(path_to_classifier, "LR_PPG_quality.pkl")
path_to_sensor_data = os.path.join(path_to_data, '1.sensor_data')
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', 'ppg')
path_to_quality_features = os.path.join(path_to_data, '3.extracted_features', 'ppg')
path_to_signal_quality = os.path.join(path_to_data, '4.predictions', 'ppg')
path_to_hr_estimate = os.path.join(path_to_data, '5.quantification', 'ppg')

In [24]:
ppg_config = PPGPreprocessingConfig()
imu_config = IMUPreprocessingConfig()
metadatas_ppg, metadatas_imu = scan_and_sync_segments(os.path.join(path_to_sensor_data, 'ppg'),
                                                       os.path.join(path_to_sensor_data, 'imu'))
preprocess_ppg_data(metadatas_ppg[0], metadatas_imu[0],
                    path_to_preprocessed_data,
                    ppg_config, imu_config)

Shape of the original data: (64775, 2) (72947, 4)
Shape of the overlapping segments: (64775, 2) (64361, 4)


(             time      green
 0        0.000000   1.143954
 1        0.033333  -9.584898
 2        0.066667 -19.145439
 3        0.100000 -26.622786
 4        0.133333 -31.575173
 ...           ...        ...
 19333  644.432719  56.004292
 19334  644.466052  27.159997
 19335  644.499385  14.340265
 19336  644.532719  16.024788
 19337  644.566052  26.286104
 
 [19338 rows x 2 columns],
              time  accelerometer_x  grav_accelerometer_x  accelerometer_y  \
 0        0.000000        -0.053078             -0.497640         0.010040   
 1        0.010000        -0.038337             -0.497667         0.058802   
 2        0.020000        -0.006824             -0.497698         0.086559   
 3        0.030000         0.009156             -0.497733         0.120855   
 4        0.040000         0.003770             -0.497772         0.159316   
 ...           ...              ...                   ...              ...   
 64454  644.539385        -0.094120             -0.879399        

In [25]:
config = SignalQualityFeatureExtractionConfig()
extract_signal_quality_features(path_to_preprocessed_data, quality_classifier_path, path_to_quality_features, config)

c:\Users\z863160\AppData\Local\pypoetry\Cache\virtualenvs\paradigma-8ps1bg0Z-py3.12\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\z863160\AppData\Local\pypoetry\Cache\virtualenvs\paradigma-8ps1bg0Z-py3.12\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 300 is greater than input length  = 3, using nperseg = 3
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
c:\Users\z863160\AppData\Local\pypoetry\Cache\virtualenvs\paradigma-8ps1bg0Z-py3.12\Lib\site-packages\scipy\signal\_spectral_py.py:600: UserWarning: nperseg = 300 is greater than input length  = 3, using nperseg = 3
  freqs, _, Pxy = _spectral_helper(x,

In [5]:
#TODO: Not fully implemented
# config = HeartRateClassificationConfig()
# signal_quality_classification(path_to_quality_features, quality_classifier_path, path_to_signal_quality, config)

In [21]:
#TODO: Not implemented
#config = HeartRateEstimationConfig()
from paradigma.heart_rate_estimation import estimate_heart_rate, estimate_heart_rate_from_raw_preprocessed
estimate_heart_rate_from_raw_preprocessed(path_to_signal_quality, path_to_preprocessed_data, path_to_hr_estimate)

NameError: name 'config' is not defined